In [217]:
import numpy as np
import creditportfolio as cp
import gaussian_copula as gc
import monte_carlo_credit as mccr
import pandas as pd
import numpy as np
import functools
from scipy import optimize

In [2]:
import numpy as np

In [261]:
initial_seed = 1414014103
ss = np.random.SeedSequence(initial_seed)
child_seq = ss.spawn(10)
generators = [np.random.default_rng(s) for s in child_seq]
m = 100
exposures_k = generators[0].gamma(shape = 10, scale = 10, size = m)
p_k = generators[1].uniform(low = 0, high = 0.1, size = m)

In [263]:
def simulate_loss(size, probs):
    defaults = generators[2].binomial(n = 1, p = probs, size = (size, m))
    L = np.dot(defaults, exposures_k)
    return L

def simulate_prob(x, n_mc, f):
    X = f(n_mc, p_k)
    return np.mean(X > x)

def is_probs(theta):
    return (p_k * np.exp(theta * exposures_k))/(1 + p_k * (np.exp(theta* exposures_k) - 1))

def psi(theta):
    return np.sum(np.log(1 + p_k * (np.exp(theta * exposures_k - 1))))

def q_expectation_loss(theta):
    return np.dot(exposures_k, is_probs(theta))

def simulate_is_prob(x, n_mc, f):
    g = lambda theta : q_expectation_loss(theta) - x
    theta_opt = optimize.newton(g, x0 =0.01)
    X = f(n_mc, is_probs(theta_opt))
    return np.mean((X > x)*np.exp(-theta_opt * X + psi(theta_opt) ))

p1 = simulate_prob(800, 1000000, simulate_loss)
p2 = simulate_is_prob(800, 100000, simulate_loss)

print(p1, p2)

0.06298 0.03724435671219112


In [251]:
simulate_prob(6000, 10000, simulate_loss)

[552.16078856 515.89796907 516.86805134 ... 118.28612648 523.97913654
 415.54584491]


0.0

In [178]:
L = simulate_loss(size = 20)
L

array([561.61595452, 517.05013674, 678.06554034, 950.58855567,
       362.66979333, 359.81444363, 149.84495763, 566.32281373,
       459.33474068, 534.26412191, 463.67629889, 244.31991277,
       451.55880634, 227.58817939, 839.93507901, 131.80039988,
       471.42920304, 608.87207132, 768.39923068, 481.70023694])